<a href="https://colab.research.google.com/github/amanbhal/data-science/blob/master/NYE_Stock_Exchange_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from pylab import rcParams

import tensorflow as tf
from keras import optimizers, Sequential
from keras.models import Model
from keras.utils import plot_model
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.layers.core import Activation, Dropout

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score

import time

from numpy.random import seed
seed(7)
from tensorflow import set_random_seed
set_random_seed(11)

from sklearn.model_selection import train_test_split

SEED = 123 #used to help randomly select the data points
DATA_SPLIT_PCT = 0.2


# load dataset
df = pd.read_csv("/content/gdrive/My Drive/ColabDatasets/NY_Stock_Market/prices-split-adjusted.csv") 

# select one company for data analysis
df = df[df['symbol']=='YHOO']

# drop unwanted columns
df = df.drop(['date', 'symbol'], axis=1)

# feature scaling
scaler = MinMaxScaler(feature_range=(0, 1))
df = scaler.fit_transform(df)

# split data into test and train
train_size = int(len(df) * 0.80)
test_size = len(df) - train_size
train, test = df[0:train_size,:], df[train_size:len(df),:]

# lookback 1 day in past
lookback = 1

def temporalize(X, lookback):
    output_X = []
    output_Y = []
    for i in range(len(X)-lookback-1):
        temp = X[i:(i+lookback), 0]
        output_X.append(temp)
        output_Y.append(X[i+lookback, 0])
    return np.array(output_X), np.array(output_Y)
  
X_train, y_train = temporalize(train,lookback)
X_test, y_test = temporalize(test,lookback)

# reshape the X_train and X_test for LSTM model
X_train = np.reshape(X_train, (X_train.shape[0], lookback, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], lookback, X_test.shape[1]))

# build the LSTM model

model = Sequential()

model.add(LSTM(
    input_dim=1,
    output_dim=50,
    return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
    100,
    return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(
    output_dim=1))
model.add(Activation('linear'))

start = time.time()
model.compile(loss='mse', optimizer='rmsprop')
print ('compilation time : ', time.time() - start)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:80: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:80: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=True, input_shape=(None, 1), units=50)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:89: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1)`
W0628 07:28:41.578960 140678651172736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



compilation time :  0.031162261962890625
